In [1]:
N = next_prime(1000)
Q = next_prime(1500)
T = N//4

_Z.<w>  = ZZ[]
R.<w>   = QuotientRing(_Z , _Z.ideal(w^N - 1))

_Q.<w>  = Integers(Q)[]
Rq.<w>  = QuotientRing(_Q , _Q.ideal(w^N - 1))

In [2]:
# Conversão polinomio -> vetor -> polinomio

def _toZ(f,p=None):
    ff = list(f)
    if p == None:
        return ff
    else:
        fp = map(lift,[Mod(a,p) for a in ff])
        return [u if u <= p//2 else u-p for u in fp ]

def _toR(vec):
    return R(vec)

def _h(f,g):
    _f = Rq(_toZ(f))
    _g = Rq(_toZ(g))
    try:
        fq = _f.inverse_of_unit()
        hq = fq * _g
        return _toR([lift(a) for a in list(hq)])
    except:
        return None

In [3]:
# Geradores

import random as rn


#def _fromS(n=N,t=T):
#    u = [rn.choice([-1,1]) for i in range(t)] + [0]*(n-t)
#    rn.shuffle(u)
#    return u

def _fromS(n=N):
    return [rn.choice([-1,0,1]) for i in range(n)]

In [4]:
def KeyGen(n=N, t=T, q=Q):
    while True:
        F = _fromS()
        G = _fromS()
        f = 1 + 3 * _toR(F)
        g = 3 * _toR(G)
        h = _h(f,g)
        if h != None:
            break
    return {'sk' : f , 'pk' : h} 

def Encrypt(plaintext,pk):
    r = _toR(_fromS())
    m = _toR(plaintext)
    return _toZ(pk*r + m, p=Q)

def Decrypt(ciphertext,sk):
    e = _toR(ciphertext)
    a = _toZ(sk*e, p=Q)
    return _toZ(_toR(a), p=3)
    
def run():
    keys   = KeyGen()
    plain  = _fromS()
    crypto = Encrypt(plain,keys['pk'])
    decryp = Decrypt(crypto,keys['sk'])
    return plain == decryp
 

In [5]:
run()

True